<a href="https://colab.research.google.com/github/deep1003/deep1003/blob/master/Patent_W2V_v2_version2_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gensim -q  # gensim 라이브러리 업그레이드 설치, '-q' 옵션은 출력 최소화


In [ ]:
# numpy 버전 1.21.5
import numpy  # numpy 라이브러리 임포트
numpy.version.version  # numpy 버전 확인


'1.21.6'

In [ ]:
# gensim 버전 4.1.2
import gensim  # gensim 라이브러리 임포트
print(gensim.__version__)  # gensim 버전 출력


4.2.0


In [ ]:
import numpy as np  # numpy 라이브러리를 np로 임포트
import pandas as pd  # pandas 라이브러리를 pd로 임포트
import os  # 운영체제와의 상호작용을 위한 os 라이브러리 임포트
from gensim.models import Word2Vec  # gensim의 Word2Vec 모델 임포트

import json  # JSON 형식의 데이터 처리를 위한 json 라이브러리 임포트
import gensim  # gensim 라이브러리 임포트
from nltk import sent_tokenize  # 문장 토큰화를 위한 sent_tokenize 함수 임포트
import nltk  # 자연어 처리 라이브러리 nltk 임포트
nltk.download('punkt')  # 'punkt' 패키지 다운로드 (문장 분리용)
nltk.download('stopwords')  # 'stopwords' 패키지 다운로드 (불용어 리스트)

from nltk.corpus import stopwords  # 불용어 처리를 위한 stopwords 임포트
stopWords = set(stopwords.words('english'))  # 영어 불용어 집합 생성
stopwords = nltk.corpus.stopwords.words('english')  # 불용어 리스트 생성

import h5py  # 대용량 데이터 저장을 위한 HDF5 형식 처리용 h5py 임포트
import tqdm  # 진행 상황을 시각적으로 보여주는 tqdm 라이브러리 임포트

import itertools  # 반복을 쉽게 처리하는 itertools 라이브러리 임포트
import gensim, logging  # gensim 및 로깅 기능을 위한 logging 라이브러리 임포트
from gensim.corpora import Dictionary  # gensim의 Dictionary 모듈 임포트
from gensim.models.tfidfmodel import TfidfModel  # TF-IDF 모델 임포트
from gensim.matutils import sparse2full  # 희소 행렬을 처리하는 함수 sparse2full 임포트

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)  # 로깅 설정 (INFO 수준)
import gc  # 메모리 관리를 위한 gc 모듈 임포트
from multiprocessing import Pool  # 병렬 처리를 위한 Pool 모듈 임포트
# from gensim.test.utils import datapath  # gensim 테스트용 유틸리티 (주석 처리됨)
from gensim.models.word2vec import Text8Corpus  # Word2Vec 모델용 Text8Corpus 임포트
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS  # Phrases 모델과 연결 단어 임포트
from gensim.parsing.preprocessing import remove_stopwords  # 불용어 제거를 위한 함수 임포트


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def ele0(x):
    return x[0]  # 입력된 리스트 또는 튜플의 첫 번째 요소를 반환

# JSON 데이터 로드
def json_l(x):
    try:
        return json.loads(x)  # JSON 문자열을 파이썬 객체로 변환
    except ValueError:
        return []  # JSON 변환에 실패하면 빈 리스트 반환

# 전처리 (문장 토크나이저 + 기본 Gensim 전처리 - 소문자화, 토큰화, (선택적) 악센트 제거)
def prepro(x):
    x = sent_tokenize(x)  # 입력 텍스트를 문장 단위로 분리
    clean = []
    for j in x:
        clean.append(gensim.utils.simple_preprocess(j, min_len=1))  # 각 문장을 소문자화하고 토큰화
    return clean  # 전처리된 문장 리스트 반환

def bigrammer(x):
    bigram = gensim.models.phrases.Phraser.load('bigram.m')  # 미리 저장된 bigram 모델 로드
    sents = []
    for i in x:
        sents.append(bigram[i])  # bigram 모델을 사용하여 입력 문장 리스트 변환
    return sents  # bigram이 적용된 문장 리스트 반환

def tfidfer(para):
    model_tfidf = gensim.models.TfidfModel.load('model_tfidf_11_2.m')  # 미리 저장된 TF-IDF 모델 로드
    docs_dict = Dictionary.load('docs_dict_11_2.d')  # 미리 저장된 사전 로드
    return model_tfidf[docs_dict.doc2bow(itertools.chain(*para))]  # 입력된 문서에 대해 TF-IDF 값을 계산하고 반환

def preprocess_1(data):
    data['appln_abstract_prepro'] = list(map(prepro, data.appln_abstract_st_r))  # 각 특허 초록에 대해 전처리 수행
    data['appln_abstract_prepro'] = list(map(json.dumps, data.appln_abstract_prepro))  # 전처리된 데이터를 JSON 형식으로 변환
    return data  # 전처리된 데이터 반환

def preprocess_2(data):
    data['no_json'] = list(map(json.loads, data['appln_abstract_prepro']))  # JSON 형식의 데이터를 파이썬 객체로 변환
    data['appln_abstract_prepro_bi'] = list(map(bigrammer, data['no_json']))  # bigram 모델 적용
    data['appln_abstract_prepro_bi'] = list(map(json.dumps, data.appln_abstract_prepro_bi))  # bigram이 적용된 데이터를 JSON 형식으로 변환
    return data  # 전처리된 데이터 반환

def preprocess_3(data):
    data['no_json_tf'] = list(map(json.loads, data['appln_abstract_prepro_bi']))  # JSON 형식의 데이터를 파이썬 객체로 변환
    data['tfidf'] = list(map(tfidfer, data['no_json_tf']))  # TF-IDF 값을 계산
    data['tfidf'] = list(map(json.dumps, data['tfidf']))  # 계산된 TF-IDF 값을 JSON 형식으로 변환
    return data  # 전처리된 데이터 반환

def get_data_path(filename):
    data_dir = '/content'  # 기본 데이터 디렉토리 설정
    path = os.path.join(data_dir, filename)  # 데이터 디렉토리와 파일 이름을 결합하여 경로 생성
    if data_dir != '.' and 'DEEP_QUANT_ROOT' in os.environ:
        path = os.path.join(os.environ['DEEP_QUANT_ROOT'], path)  # 환경 변수를 기반으로 경로 재설정
    return path  # 최종 경로 반환

def abstract_generator_nonmult(df):
    go = 'OK'
    while go == 'OK':
        try:
            i = df['appln_abstract_prepro_bi']  # 전처리된 특허 초록 데이터 가져오기
        except StopIteration:
            return  # 데이터가 더 이상 없으면 종료
        if i.isnull().sum() > 0:
            i = list(filter(lambda a: a != None, i))  # None 값을 제외한 리스트 필터링
            go = 'STOP'  # 처리 중지
        if i.isnull().sum() < 0:
            i = [x[0] for x in i]  # 각 요소의 첫 번째 값만 가져오기
        j = [json_l(x) for x in i]  # JSON 데이터를 파이썬 객체로 변환
        for h in j:
            try:
                yield list(itertools.chain(*h))  # 중첩된 리스트를 하나의 리스트로 병합하여 반환
            except StopIteration:
                return  # 데이터가 더 이상 없으면 종료


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/AI-Growth-Lab/Patent_p2p_similarity_w2v/main/patent_dtatset_sample_1k.csv')
# 'https://raw.githubusercontent.com/AI-Growth-Lab/Patent_p2p_similarity_w2v/main/patent_dtatset_sample_1k.csv'에서 CSV 파일을 읽어와 데이터프레임으로 로드


In [ ]:
tokens_without_sw = [remove_stopwords(text) for text in data.appln_abstract]  # 특허 초록에서 불용어를 제거한 토큰 리스트 생성
data['appln_abstract_st_r'] = tokens_without_sw  # 불용어가 제거된 토큰 리스트를 데이터프레임의 새로운 열 'appln_abstract_st_r'에 추가


In [ ]:
data_1 = preprocess_1(data)  # preprocess_1 함수를 사용하여 데이터 전처리 수행
data_1.to_csv('data.csv')  # 전처리된 데이터를 'data.csv' 파일로 저장


In [ ]:
sentences = Text8Corpus(get_data_path('data.csv'))  # 'data.csv' 파일을 Text8Corpus 형식으로 불러와 문장 리스트 생성


In [ ]:
bigram = Phrases(sentences, min_count=1, threshold=1, connector_words=ENGLISH_CONNECTOR_WORDS)
# 주어진 문장 리스트에서 자주 함께 등장하는 단어 쌍을 찾기 위해 Phrases 모델 생성
# min_count=1: 단어 쌍이 최소 한 번 이상 등장해야 함
# threshold=1: 단어 쌍이 연결되는 기준값 설정
# connector_words=ENGLISH_CONNECTOR_WORDS: 연결 단어로 사용될 영어 단어 목록 설정


In [ ]:
bigram = gensim.models.phrases.Phraser(bigram)  # 생성된 Phrases 모델을 메모리 효율적인 Phraser 객체로 변환


In [ ]:
bigram.save('bigram.m')  # bigram Phraser 객체를 'bigram.m' 파일로 저장


In [ ]:
data_2 = preprocess_2(data_1)  # preprocess_2 함수를 사용하여 데이터 전처리 2단계 수행


In [ ]:
data_2.to_csv('data.csv')  # 전처리된 2단계 데이터를 'data.csv' 파일로 저장


In [ ]:
model_with_phrases = Word2Vec(
    sentences=sentences,  # 학습할 문장 리스트
    vector_size=300,  # 각 단어를 300차원 벡터로 변환
    window=80,  # 학습할 때 고려할 문맥의 크기 (앞뒤 80개 단어)
    min_count=1,  # 최소 등장 횟수가 1 이상인 단어만 학습
    workers=14  # 병렬 처리를 위해 사용할 CPU 코어 수
)


In [ ]:
model_with_phrases.save('w2v.m')  # 학습된 Word2Vec 모델을 'w2v.m' 파일로 저장


In [ ]:
voc = model_with_phrases.wv.index_to_key  # 학습된 Word2Vec 모델에서 단어 리스트(어휘)를 가져옴


In [ ]:
# Gensim Dictionary 생성
docs_dict = Dictionary([voc])  # Word2Vec 모델에서 가져온 단어 리스트를 사용하여 Gensim Dictionary 생성
docs_dict.compactify()  # Dictionary에서 불필요한 빈 공간을 제거하여 메모리 사용을 최적화
docs_dict.save('docs_dict_11_2.d')  # 생성된 Dictionary를 'docs_dict_11_2.d' 파일로 저장


In [ ]:
model_tfidf = TfidfModel((docs_dict.doc2bow(x) for x in sentences), id2word=docs_dict)
# 문장 리스트를 사용하여 TF-IDF 모델 생성, 각 문장을 BOW 형식으로 변환하여 입력
# id2word=docs_dict는 BOW 형식의 ID를 실제 단어로 매핑하는 데 사용됨

model_tfidf.save('model_tfidf_11_2.m')  # 생성된 TF-IDF 모델을 'model_tfidf_11_2.m' 파일로 저장


In [ ]:
data_3 = preprocess_3(data_2)  # preprocess_3 함수를 사용하여 데이터 전처리 3단계 수행
data_3.to_csv('data.csv')  # 전처리된 3단계 데이터를 'data.csv' 파일로 저장
iterator = abstract_generator_nonmult(data_3)  # 전처리된 데이터를 기반으로 생성된 iterator 생성


In [ ]:
# 문서 행렬 생성 (TF-IDF 가중치 적용)
docs_vecs = (sparse2full(c, len(docs_dict)) for c in (model_tfidf[docs_dict.doc2bow(x)] for x in iterator))
# iterator를 통해 생성된 각 문서에 대해 BOW 형식으로 변환한 후, TF-IDF 모델을 적용하여 가중치 계산
# 계산된 가중치 벡터를 sparse2full 함수를 통해 밀집 벡터로 변환하고, 전체 문서 사전의 길이로 맞춤


In [ ]:
# 선택된 단어 임베딩 벡터 생성
emb_vecs_selftrained = np.vstack([model_with_phrases.wv[docs_dict.get(i)] for i in range(len(docs_dict))])
# 각 단어에 대해 학습된 Word2Vec 모델에서 임베딩 벡터를 가져와 수직으로 쌓아 하나의 배열로 만듦

n_abstracts = len(data_2)  # 데이터에서 특허 초록의 개수를 계산
h5f = h5py.File('docvecs_23_4_test.h5', 'a')  # HDF5 파일을 열거나 생성하여 'a' 모드로 사용 ('a'는 파일이 없으면 생성하고, 있으면 열어 추가 작업)
dataset = h5f.create_dataset('weighted_tfidf_docvecs', (n_abstracts, 300))
# 'weighted_tfidf_docvecs'라는 이름으로 (특허 초록 수, 300차원) 크기의 데이터셋을 생성


In [ ]:
# 문서 벡터 생성
pbar = tqdm.tqdm(total=n_abstracts)  # 진행 상황을 시각적으로 표시하기 위한 진행 막대 설정
start = 0

# while 루프 대신 테스트를 위해 1회 반복하도록 설정
for i in range(1):
    a = np.vstack(next(docs_vecs) for _ in range(n_abstracts))  # TF-IDF 가중치가 적용된 문서 벡터 생성
    pbar.update(n_abstracts)  # 진행 막대를 업데이트하여 n_abstracts만큼 증가
    b = np.dot(a, emb_vecs_selftrained)  # 생성된 문서 벡터와 임베딩 벡터를 내적하여 최종 문서 벡터 생성
    end = start + len(b)  # 데이터셋에 추가될 범위 설정
    dataset[start:end] = b  # 생성된 벡터를 데이터셋에 저장
    start = end  # 시작 인덱스 업데이트

pbar.close()  # 진행 막대 닫기
h5f.close()  # HDF5 파일 닫기


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  
100%|██████████| 1000/1000 [00:01<00:00, 576.61it/s]


In [ ]:
h5f = h5py.File('docvecs_23_4_test.h5', 'r')  # 'docvecs_23_4_test.h5' 파일을 읽기 모드('r')로 열기
dataset = h5f['weighted_tfidf_docvecs']  # 파일에서 'weighted_tfidf_docvecs' 데이터셋 가져오기
dataset[1]  # 데이터셋에서 두 번째 문서 벡터 반환


array([-0.05205272, -0.461966  ,  0.4014555 ,  0.5138215 , -0.31621286,
       -0.62124974, -0.25785035,  1.3022149 ,  0.14012174, -0.9412803 ,
        1.035978  ,  0.09522733, -1.055949  ,  1.2118465 , -0.43018413,
       -0.686788  ,  0.9686846 , -0.58684635, -0.1631586 , -0.9776744 ,
        0.01906457,  1.0731976 ,  0.8642982 ,  1.264511  , -0.2250568 ,
       -0.54605645, -0.51214844,  0.80090696, -0.6588723 , -0.67597836,
        1.7397033 , -0.19973068,  1.0296581 , -1.1289895 ,  0.7445261 ,
       -0.9251042 , -0.08644897, -1.0327184 ,  0.5954751 , -0.2610794 ,
       -0.6402455 , -0.17841025, -0.44318157, -1.2373002 , -0.13993335,
        0.03662474, -0.7218325 ,  0.11164966,  0.02642337, -0.08000155,
        0.0187408 ,  0.8784889 , -0.5367489 ,  0.4813019 ,  0.00459789,
       -0.35290617,  0.604631  , -0.6003416 , -0.41737077,  0.48500222,
        0.6599798 , -0.41417733,  0.49715832,  1.3559908 , -0.64288056,
        0.9960632 , -0.44202578, -0.3869542 , -0.92245907,  0.20

In [ ]:
import scipy  # scipy 라이브러리 임포트
scipy.spatial.distance.cosine(dataset[9], dataset[8])  # 데이터셋의 10번째 문서와 9번째 문서 사이의 코사인 거리 계산


0.5613730847835541

In [ ]:
# Annoy 라이브러리 설치 및 임포트
!pip install annoy  # Annoy 라이브러리 설치
from annoy import AnnoyIndex  # AnnoyIndex 클래스 임포트

t = AnnoyIndex(dataset.shape[1])  # 인덱싱할 아이템 벡터의 길이를 설정하여 AnnoyIndex 객체 생성


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 4.0 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=395183 sha256=b7ff3fb240c88dc4a3939c90c3357c98d50e21930d58966113a89689fb149e1d
  Stored in directory: /root/.cache/pip/wheels/81/94/bf/92cb0e4fef8770fe9c6df0ba588fca30ab7c306b6048ae8a54
Successfully built annoy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  after removing the cwd from sys.path.


In [ ]:
t.on_disk_build('vecs_build.annoy')  # Annoy 인덱스를 'vecs_build.annoy' 파일에 디스크에 저장하며 빌드하도록 설정


True

In [ ]:
for i in tqdm.tqdm(range(dataset.shape[0])):  # 데이터셋의 크기만큼 진행 상황을 표시하며 반복
    t.add_item(i, dataset[i])  # 각 문서 벡터를 Annoy 인덱스에 추가


100%|██████████| 1000/1000 [00:00<00:00, 8882.40it/s]


In [ ]:
t.build(20)  # Annoy 인덱스를 20개의 트리로 빌드하여 효율적인 근접 이웃 검색을 가능하게 함


True

In [ ]:
tt = AnnoyIndex(300)  # 300차원 벡터를 위한 새로운 AnnoyIndex 객체 생성
tt.load('vecs_build.annoy', prefault=False)  # 'vecs_build.annoy' 파일에서 인덱스를 로드, prefault=False로 메모리 사용 최적화


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


True

In [ ]:
simH5 = h5py.File('similarities_23_2.h5', 'a')  # 'similarities_23_2.h5' 파일을 열거나 생성하여 'a' 모드로 사용 ('a' 모드는 파일이 없으면 생성, 있으면 열어 추가 작업 가능)


In [ ]:
dataset = simH5.create_dataset(
    'sims_50',  # 데이터셋 이름
    (10000000, 3),  # 초기 크기 (1000만 행, 3 열)
    maxshape=(None, 3),  # 최대 크기 설정 (행의 수는 제한 없음, 3 열)
    compression="gzip"  # gzip 압축을 사용하여 저장 공간 절약
)


In [ ]:
round(t.get_n_items() / 10)  # Annoy 인덱스에 저장된 아이템 수를 10으로 나눈 값의 반올림 값 반환


100

In [ ]:
pbar = tqdm.tqdm(total=tt.get_n_items())  # 진행 상황을 시각적으로 표시하기 위한 진행 막대 설정

blocks = []  # 블록을 저장할 리스트 초기화
start = 0  # 시작 인덱스 초기화
end = 0  # 종료 인덱스 초기화
counter = 0  # 카운터 초기화

for i in range(tt.get_n_items()):  # Annoy 인덱스에 저장된 모든 아이템에 대해 반복
    j = 100  # 초기 검색 범위 설정
    tresh = 0.0  # 초기 임계값 설정
    while tresh < 0.5:  # 임계값이 0.5 이상이 될 때까지 반복
        y = tt.get_nns_by_item(i, j, search_k=50, include_distances=True)  # 가까운 이웃 검색
        tresh = y[1][-1]  # 가장 먼 거리의 이웃 설정
        j += 50  # 검색 범위 확장
        if j >= 500:  # 검색 범위가 500을 넘으면 중지
            break
    block = np.array(list(zip([i for _ in range(len(y[0]))], y[0], y[1])))  # 결과를 배열로 변환하여 블록 생성
    end = end + len(block)  # 종료 인덱스 업데이트
    blocks.append(block)  # 생성된 블록을 블록 리스트에 추가
    counter += 1  # 카운터 증가
    if counter == 1000:  # 1000번마다 데이터셋에 저장
        counter = 0  # 카운터 초기화
        blocks_np = np.vstack(blocks)  # 블록들을 수직으로 쌓아 하나의 배열로 변환
        if end >= dataset.shape[0]:  # 종료 인덱스가 데이터셋 크기를 넘어서면
            dataset.resize((end + (end - dataset.shape[0]), 3))  # 데이터셋 크기를 조정
        dataset[start:end] = blocks_np  # 데이터셋에 블록 저장
        blocks = []  # 블록 리스트 초기화
        gc.collect()  # 가비지 컬렉션 호출
        start = end  # 시작 인덱스 업데이트
        pbar.update(1000)  # 진행 막대 업데이트

simH5.close()  # HDF5 파일 닫기


100%|██████████| 1000/1000 [00:00<00:00, 1143.01it/s]

In [ ]:
def lookup(x):
    y = tree.get_nns_by_item(x, j, search_k=5, include_distances=True)  # 특정 아이템 x에 대해 가장 가까운 이웃 검색
    return np.array(list(zip([x for _ in range(len(y[0]))], y[0], y[1])))  # 결과를 배열로 변환하여 (아이템 x, 이웃 아이템, 거리)의 튜플 리스트 반환
